In [6]:
!apt-get update
!apt-get install openjdk-11-jdk -y

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.30+7-1ubuntu1~22.04).
0 upgraded, 0 newly i

In [7]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.ml.feature import *
from pyspark.ml.regression import *

In [8]:
df_video = spark.read.parquet('videos-comments-tratados.snappy.parquet', header=True, inferSchema=True) # Lendo o arquivo
df_video.show(5)
df_video.printSchema()

+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments| Views|Interaction|Year|             Comment|Sentiment|Likes Comment|
+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Let's not forget ...|        1|           95|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Here in NZ 50% of...|        0|           19|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|I will forever ac...|        2|          161|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Whenever I go to ...|        0|            8|
|wAZZ-UWGVHI|Apple P

In [9]:
spark = SparkSession.builder.getOrCreate()

In [10]:
df_video = df_video.dropna()

In [12]:
df_video = df_video.withColumn('Month', month(col('Published At'))) # Adicionando a coluna mês e atribuindo de volta a df_video
df_video.show(5)

+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+
|   Video ID|               Title|Published At|Keyword|Likes|Comments| Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|
+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Let's not forget ...|        1|           95|    8|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Here in NZ 50% of...|        0|           19|    8|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|I will forever ac...|        2|          161|    8|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Whenever I go to ...|        0

In [13]:
string_indexer = StringIndexer(inputCol='Keyword', outputCol='Keyword Index') # Instanciando StringIndexer
modelo_indexer = string_indexer.fit(df_video) # O StringIndexer aprende a indexação a partir do DataFrame df_video
df_video_indexado = modelo_indexer.transform(df_video) # Transforma o DataFrame usando o modelo
df_video_indexado.show()

+-----------+--------------------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+-----+-------------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments|  Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|
+-----------+--------------------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+-----+-------------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Let's not forget ...|        1|           95|    8|         10.0|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Here in NZ 50% of...|        0|           19|    8|         10.0|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|I will forever ac...|        2|          161|    8|         10.0|
|wAZZ-UWGVHI|Apple Pay Is Kill...|

In [14]:
df_video_indexado = df_video_indexado.withColumn("Year", col("Year").cast("int"))
df_video_indexado = df_video_indexado.withColumn("Month", col("Month").cast("int"))

In [15]:
montar_vetor = VectorAssembler(
  inputCols=["Likes", "Views", "Year", "month", "Keyword Index"],
  outputCol='Features'
) # Criando o Vetor

df_video_caracteristicas = montar_vetor.transform(df_video_indexado)
df_video_caracteristicas.show()
df_video_caracteristicas.printSchema()

+-----------+--------------------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments|  Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|            Features|
+-----------+--------------------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Let's not forget ...|        1|           95|    8|         10.0|[3407.0,135612.0,...|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Here in NZ 50% of...|        0|           19|    8|         10.0|[3407.0,135612.0,...|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|

In [16]:
scaler = MinMaxScaler(inputCol='Features', outputCol='Features Normal')
modelo_scaler = scaler.fit(df_video_caracteristicas)

df_video_normalizado = modelo_scaler.transform(df_video_caracteristicas)
df_video_normalizado.select('Features Normal').show(truncate=False)

+------------------------------------------------------------------------+
|Features Normal                                                         |
+------------------------------------------------------------------------+
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[2.072291978642988E-4,6.31492585237833E-5,1.0,0.6363636363636364,0.25]  |
|[0.00466873762089814,8.1

In [17]:
pca = PCA(k=1, inputCol='Features Normal', outputCol='Features PCA')
modelo_pca = pca.fit(df_video_normalizado)
df_video_pca = modelo_pca.transform(df_video_normalizado)

df_video_pca.show(truncate=False)

+-----------+--------------------------------------------------------------------------------------------------+------------+-------+-----+--------+-------+-----------+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-------------+-----+-------------+-----------------------------------+------------------------------------------------------------------------+---------------------+
|Video ID   |Title                                                                                             |Published At|Keyword|Likes|Comments|Views  |Interaction|Year|Comment           

In [18]:
train_data, test_data = df_video_pca.randomSplit([0.8, 0.2], seed=42)
print(f'Treino: {train_data.count()}, Teste: {test_data.count()}')


Treino: 12114, Teste: 2956


In [19]:
regressao_linear = LinearRegression(featuresCol='Features Normal', labelCol='Comments')
modelo_lr = regressao_linear.fit(train_data)

avaliar_test = modelo_lr.evaluate(test_data)
print('RMSE (Erro Quadrático Médio da Raiz:).:', avaliar_test.rootMeanSquaredError)
print('R2 (Coeficiente de Determinação): ', avaliar_test.r2)

RMSE (Erro Quadrático Médio da Raiz:).: 15709.606919129044
R2 (Coeficiente de Determinação):  0.8420584208295412


In [20]:
df_video_pca.write.option('header', 'true').parquet('videos-preparados-parquet')